# UNIPROT CANCER

In [2]:
!pip install biopython

     ---------------------------------------- 2.7/2.7 MB 3.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import os
import subprocess
from Bio import SeqIO
from Bio.Blast import NCBIWWW
import glob
import re
import zipfile
import concurrent.futures
import shutil
import pickle
import random

In [4]:
cancer_proteins_uniprot = pd.read_csv('D:/CODING/Code_new/Data Bahan/uniprot2seq_cancer_from_uniprot.tsv', sep='\t')
cancer_proteins_uniprot

,Entry,Protein names,Protein families,Sequence
0,A0AVK6,Transcription factor E2F8 (E2F-8),E2F/DP family,MENEKENLFCEPHKRGLMKTPLKESTTANIVLAEIQPDFGPLTTPT...
1,A0AVT1,Ubiquitin-like modifier-activating enzyme 6 (U...,Ubiquitin-activating E1 family,MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYSR...
2,A0FGR8,Extended synaptotagmin-2 (E-Syt2) (Chr2Syt),Extended synaptotagmin family,MTANRDAALSSHRHPGCAQRPRTPTFASSSQRRSAFGFDDGNFPGL...
3,A0JLT2,Mediator of RNA polymerase II transcription su...,Mediator complex subunit 19 family,MENFTALFGAQADPPPPPTALGFGPGKPPPPPPPPAGGGPGTAPPP...
4,A0JNW5,Bridge-like lipid transfer protein family memb...,NaN,MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVL...
...,...,...,...,...
7787,Q8WZ26,Putative uncharacterized protein PP6455,NaN,MCEQLVCIFTYAYLKINLTKALGHHNPSYPVARAQPQTMFPSASEC...
7788,Q8WZA8,Putative gastric cancer-related gene 224 prote...,NaN,MIPGNPSPGADLAVSKHFFSLSWFCGLLLLESKQK
7789,Q96MH7,Uncharacterized protein C5orf34,NaN,MAAELRMILYEDDSVQVQYVDGSTLQLSPCGSEFLFEKSPPVSAHP...
7790,Q9BZK8,Ovarian cancer-related protein 1,NaN,MPVAPSNHCDNQCPHIFSKALVVSVAPSPPRDKPAPYTFTDVSSLC...


In [5]:
protein_entry_uniprot = cancer_proteins_uniprot['Entry']

# DRUGBANK 

In [6]:
drugbank_peptides = pd.read_csv('D:/CODING/Code_new/Data Bahan/DrugBank Full DTI.csv')
drugbank_peptides

,dg_id,dg_name,dg_synm,dg_kingdom,dg_superclass,dg_interactions,dg_categories,dg_sequence,dg_ext_id,dg_pathways,target_name,target_uniprot,target_gene_name,action,cell_loc
0,DB00001,Lepirudin,"[Leu1, Thr2]-63-desulfohirudin;Desulfatohirudi...",Organic Compounds,Organic Acids,DB06605;DB06695;DB01254;DB01609;DB01586;DB0212...,"Amino Acids, Peptides, and Proteins;Anticoagul...",>DB00001 sequence\nLTYTDCTESGQNLCLCEGSNVCGQGNK...,Drugs Product Database (DPD):11916;PubChem Sub...,Lepirudin Action Pathway,Prothrombin,P00734,F2,inhibitor,Secreted
1,DB00002,Cetuximab,Cetuximab;Cétuximab;Cetuximabum,Organic Compounds,Organic Acids,DB00255;DB00269;DB00286;DB00655;DB00783;DB0089...,"Amino Acids, Peptides, and Proteins;Antibodies...",>Cetuximab heavy chain\nQVQLKQSGPGLVQPSQSLSITC...,Drugs Product Database (DPD):13175;PubChem Sub...,Cetuximab Action Pathway,Epidermal growth factor receptor,P00533,EGFR,binder,Cell membrane
2,DB00002,Cetuximab,Cetuximab;Cétuximab;Cetuximabum,Organic Compounds,Organic Acids,DB00255;DB00269;DB00286;DB00655;DB00783;DB0089...,"Amino Acids, Peptides, and Proteins;Antibodies...",>Cetuximab heavy chain\nQVQLKQSGPGLVQPSQSLSITC...,Drugs Product Database (DPD):13175;PubChem Sub...,Cetuximab Action Pathway,Low affinity immunoglobulin gamma Fc region re...,O75015,FCGR3B,binder,Cell membrane
3,DB00002,Cetuximab,Cetuximab;Cétuximab;Cetuximabum,Organic Compounds,Organic Acids,DB00255;DB00269;DB00286;DB00655;DB00783;DB0089...,"Amino Acids, Peptides, and Proteins;Antibodies...",>Cetuximab heavy chain\nQVQLKQSGPGLVQPSQSLSITC...,Drugs Product Database (DPD):13175;PubChem Sub...,Cetuximab Action Pathway,Complement C1q subcomponent subunit A,P02745,C1QA,binder,Secreted
4,DB00002,Cetuximab,Cetuximab;Cétuximab;Cetuximabum,Organic Compounds,Organic Acids,DB00255;DB00269;DB00286;DB00655;DB00783;DB0089...,"Amino Acids, Peptides, and Proteins;Antibodies...",>Cetuximab heavy chain\nQVQLKQSGPGLVQPSQSLSITC...,Drugs Product Database (DPD):13175;PubChem Sub...,Cetuximab Action Pathway,Complement C1q subcomponent subunit B,P02746,C1QB,binder,Secreted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15036,DB17635,Nedosiran,DCR-PHXC free acid;Nedosiran,Organic Compounds,Organic Acids,NaN,NaN,NaN,Wikipedia:Nedosiran,NaN,L-lactate dehydrogenase A chain,P00338,LDHA,antisense oligonucleotide,Cytoplasm
15037,DB17851,Flotufolastat F-18,18F-rhPSMA-7.3;Flotufolastat (18F);Flotufolast...,Organic Compounds,Organic Acids,DB00499;DB00665;DB01128;DB04839;DB08899;DB1185...,Diagnostic Radiopharmaceuticals;Fluorine Radio...,NaN,RxCUI:2637946,NaN,Glutamate carboxypeptidase 2,Q04609,FOLH1,binder,Cell membrane
15038,DB18236,(R)-9b,NaN,Organic Compounds,Organic Acids,NaN,NaN,NaN,NaN,NaN,Activated CDC42 kinase 1,Q07912,TNK2,inhibitor,Cell membrane
15039,DB18680,Lovotibeglogene autotemcel,Lovo-cel,Organic Compounds,Organic Acids,NaN,Autologous Cultured Cell;Autologous Hematopoie...,NaN,Wikipedia:Lovotibeglogene_autotemcel,NaN,Hemoglobin subunit beta,NaN,NaN,gene replacement,NaN


In [7]:
drugbank_peptides = drugbank_peptides[['dg_id', 'dg_sequence', 'target_uniprot']]
drugbank_peptides = drugbank_peptides.dropna()
drugbank_peptides

,dg_id,dg_sequence,target_uniprot
0,DB00001,>DB00001 sequence\nLTYTDCTESGQNLCLCEGSNVCGQGNK...,P00734
1,DB00002,>Cetuximab heavy chain\nQVQLKQSGPGLVQPSQSLSITC...,P00533
2,DB00002,>Cetuximab heavy chain\nQVQLKQSGPGLVQPSQSLSITC...,O75015
3,DB00002,>Cetuximab heavy chain\nQVQLKQSGPGLVQPSQSLSITC...,P02745
4,DB00002,>Cetuximab heavy chain\nQVQLKQSGPGLVQPSQSLSITC...,P02746
...,...,...,...
15001,DB16703,>Human Plasminogen:\nMEHKEVVLLLLLFLKSGQGEPLDDY...,O75116
15002,DB16703,>Human Plasminogen:\nMEHKEVVLLLLLFLKSGQGEPLDDY...,Q13464
15004,DB16708,>Cipaglucosidase alfa sequence\nQQGASRPGPRDAQA...,P11717
15017,DB16756,>Heavy Chain\nQITLKESGPTLVKPTQTLTLTCTFSGFSLSIS...,P23458


In [8]:
db_peptides = drugbank_peptides.copy()

In [9]:
len_df = len(db_peptides)
new_rows = pd.DataFrame()
for index, row in db_peptides.iterrows():
    sequence = row['dg_sequence']
    if sequence.startswith(';'):
        sequence = sequence[1:] 
    parts = sequence.split(';')  
    for part in parts:
        part = part.strip() 
        if part.startswith('> '):
            part = part[2:]  
        drug_type, sequence = part.split('\n', 1)
        new_row = row.copy()
        new_row['dg_sequence'] = sequence
        new_row['drug_type'] = drug_type
        new_rows = pd.concat([new_rows, new_row.to_frame().T], ignore_index=True)

db_peptides = pd.concat([db_peptides, new_rows], ignore_index=True)

db_peptides = db_peptides.iloc[len_df:, :]

db_peptides['drug_type'] = db_peptides['drug_type'].str.replace('>', '').str.replace('sequence', '', case=False)
db_peptides['dg_sequence'] = db_peptides['dg_sequence'].replace(r'\s+', '', regex=True)

db_peptides.reset_index(drop=True, inplace=True)

In [10]:
db_peptides

,dg_id,dg_sequence,target_uniprot,drug_type
0,DB00001,LTYTDCTESGQNLCLCEGSNVCGQGNKCILGSDGEKNQCVTGEGTP...,P00734,DB00001
1,DB00002,QVQLKQSGPGLVQPSQSLSITCTVSGFSLTNYGVHWVRQSPGKGLE...,P00533,Cetuximab heavy chain
2,DB00002,DILLTQSPVILSVSPGERVSFSCRASQSIGTNIHWYQQRTNGSPRL...,P00533,Cetuximab light chain
3,DB00002,QVQLKQSGPGLVQPSQSLSITCTVSGFSLTNYGVHWVRQSPGKGLE...,O75015,Cetuximab heavy chain
4,DB00002,DILLTQSPVILSVSPGERVSFSCRASQSIGTNIHWYQQRTNGSPRL...,O75015,Cetuximab light chain
...,...,...,...,...
14906,DB16708,QQGASRPGPRDAQAHPGRPRAVPTQCDVPPNSRFDCAPDKAITQEQ...,P11717,Cipaglucosidase alfa
14907,DB16756,QITLKESGPTLVKPTQTLTLTCTFSGFSLSISGVGVGWLRQPPGKA...,P23458,Heavy Chain
14908,DB16756,QSALTQPASVSGSPGQSITISCTATSSDVGDYNYVSWYQQHPGKAP...,P23458,Light Chain
14909,DB18704,TPLGPASSLPQSFLLKCLEQVRKIQGDGAALQEKLCATYKLCHPEE...,Q99062,SUBUNIT_1


In [11]:
db_peptides = db_peptides[db_peptides['dg_sequence'].str.len() <= 50]
db_peptides

,dg_id,dg_sequence,target_uniprot,drug_type
37,DB00010,YADAIFTNSYRKVLGQLSARKLLQDIMSRQ,Q02643,DB00010
55,DB00017,CSNLSTCVLGKLSQELHKLQTYPRTNTGSGTP,P30988,DB00017
109,DB00030,GIVEQCCTSICSLYQLENYCN,P06213,A chain
110,DB00030,FVNQHLCGSHLVEALYLVCGERGFFYTPKT,P06213,B chain
111,DB00030,GIVEQCCTSICSLYQLENYCN,P08069,A chain
...,...,...,...,...
14350,DB11995,AIYPGNGDTSYNQKFKG,Q8TCC7,CDR H2 (SEQ ID 5)
14351,DB11995,VVYYSNSYWYFDV,Q8TCC7,CDR H3 (SEQ ID 6)
14793,DB14939,RRXCYXKKPYRXCR,P61073,Motixafortide
14805,DB15171,YXEGTFTSDYSIXLDKIAQKAFVQWLIAGGPSSGAPPPS,P43220,Tirzepatide protein


In [12]:
len(db_peptides['dg_sequence'].unique())
print(db_peptides['dg_sequence'].unique())

['YADAIFTNSYRKVLGQLSARKLLQDIMSRQ' 'CSNLSTCVLGKLSQELHKLQTYPRTNTGSGTP'
 'GIVEQCCTSICSLYQLENYCN' 'FVNQHLCGSHLVEALYLVCGERGFFYTPKT'
 'HSQGTFTSDYSKYLDSRRAQDFVQWLMNT' 'FVNQHLCGSHLVEALYLVCGERGFFYTKPT'
 'GIVEQCCTSICSLYQLENYCG' 'FVNQHLCGSHLVEALYLVCGERGFFYTPKTRR' 'CYIQNCPLG'
 'HGEGTFTSDLSKQMEEEAVRLFIEWLKNGGPSSGAPPPS'
 'SYSMEHFRWGKPVGKKRRPVKVYPDGAEDQLAEAFPLEF'
 'FVNQHLCGSHLVEALYLVCGERGFFYTDKT' 'FVNQHLCGSHLVEALYLVCGERGFFYTPK'
 'FVKQHLCGSHLVEALYLVCGERGFFYTPET' 'GGGCYFQNCPKG' 'KLLLLKLLLLKLLLLKLLLLK'
 'SPKMVQGSGCFGRKMDRISSSSGLGCKVLRRH' 'PGDSTRKCMDLKGNK'
 'AVSEHQLLHDKGKSIQDLRRRELLEKLLXKLHTA' 'IEGPTLRQWLAARA'
 'MKHLLLLLLCVFLVKSQGVNDNEEGFFS' 'CUCCAACAUCAAGGAAGAUGGCAUUUCUAG'
 'SVSEIQLMHNLGKHLNSMERVEWLRKKLQDVHNF' 'HAEGTFTSDVSSYLEGQAAKEEFIAWLVRGRG'
 'CGNLSTCMLGTYTQDFNKFHTFPQTAIGVGAP'
 'YADAIFTNSYRKVLGQLSARKLLQDIMSRQQGESNQERGARARL' 'APSFDCGKPQVEPKKCPGR'
 'HGDGSFSDEMNTILDNLAARDFINWLIQTKITD'
 'HGEGTFTSDLSKQMEEEAVRLFIEWLKNGGPSSGAPPSKKKKKK'
 'HSDGTFTSELSRLREGARLQRLLQGLV' 'RASSSVSYMH' 'APSNLAS' 'QQWSFNPPT'
 'GYTF

In [13]:
db_peptides = db_peptides.drop_duplicates(subset=['dg_sequence', 'target_uniprot'])
db_peptides

,dg_id,dg_sequence,target_uniprot,drug_type
37,DB00010,YADAIFTNSYRKVLGQLSARKLLQDIMSRQ,Q02643,DB00010
55,DB00017,CSNLSTCVLGKLSQELHKLQTYPRTNTGSGTP,P30988,DB00017
109,DB00030,GIVEQCCTSICSLYQLENYCN,P06213,A chain
110,DB00030,FVNQHLCGSHLVEALYLVCGERGFFYTPKT,P06213,B chain
111,DB00030,GIVEQCCTSICSLYQLENYCN,P08069,A chain
...,...,...,...,...
14350,DB11995,AIYPGNGDTSYNQKFKG,Q8TCC7,CDR H2 (SEQ ID 5)
14351,DB11995,VVYYSNSYWYFDV,Q8TCC7,CDR H3 (SEQ ID 6)
14793,DB14939,RRXCYXKKPYRXCR,P61073,Motixafortide
14805,DB15171,YXEGTFTSDYSIXLDKIAQKAFVQWLIAGGPSSGAPPPS,P43220,Tirzepatide protein


# PDB

In [14]:
def fasta_to_dataframe(fasta_file):
    records = list(SeqIO.parse(fasta_file, "fasta"))

    data = {'ID': [record.id for record in records],
            'Description': [record.description for record in records],
            'Sequence': [str(record.seq) for record in records]}

    df = pd.DataFrame(data)
    return df

pdb_seqres = fasta_to_dataframe('D:/CODING/Code_new/Data Bahan/PDB/pdb_seqres.txt')
pdb_seqres = pdb_seqres[['ID', 'Sequence']]
# filter peptides only
pdb_seqres = pdb_seqres[pdb_seqres['Sequence'].str.len() < 50]

In [15]:
pdb_seqres

,ID,Sequence
0,100d_A,CCGGCGCCGG
1,100d_B,CCGGCGCCGG
2,101d_A,CGCGAATTCGCG
3,101d_B,CGCGAATTCGCG
5,102d_A,CGCAAATTTGCG
...,...,...
825718,9ins_B,FVNQHLCGSHLVEALYLVCGERGFFYTPKA
825725,9lpr_P,XAAPL
825727,9mht_C,CCATGCGCTGAC
825728,9mht_D,GTCAGNGCATGG


# SIFTS (UNIPROT)

In [16]:
pdb_chain_uniprot = pd.read_csv('D:/CODING/Code_new/Data Bahan/Uniprot (SIFTS)/pdb_chain_uniprot.tsv', sep='\t')
pdb_chain_uniprot['PDB_ID'] = pdb_chain_uniprot['PDB'] + '_' + pdb_chain_uniprot['CHAIN']
pdb_chain_uniprot = pdb_chain_uniprot[['PDB_ID', 'SP_PRIMARY', 'RES_BEG', 'RES_END', 'PDB_BEG', 'PDB_END', 'SP_BEG', 'SP_END']]

In [17]:
pdb_chain_uniprot

,PDB_ID,SP_PRIMARY,RES_BEG,RES_END,PDB_BEG,PDB_END,SP_BEG,SP_END
0,101m_A,P02185,1,154,0,153,1,154
1,102l_A,P00720,1,40,1,40,1,40
2,102l_A,P00720,42,165,41,None,41,164
3,102m_A,P02185,1,154,0,153,1,154
4,103l_A,P00720,1,40,1,None,1,40
...,...,...,...,...,...,...,...,...
764861,9xia_A,P24300,1,388,1,None,1,388
764862,9xim_A,P12851,1,393,None,394,2,394
764863,9xim_B,P12851,1,393,None,394,2,394
764864,9xim_C,P12851,1,393,None,394,2,394


In [18]:
pdb_chain_uniprot_cancer = pd.merge(pdb_chain_uniprot, protein_entry_uniprot, left_on='SP_PRIMARY', right_on='Entry', how='inner')
pdb_chain_uniprot_cancer = pdb_chain_uniprot_cancer[['PDB_ID', 'SP_PRIMARY', 'RES_BEG', 'RES_END', 'PDB_BEG', 'PDB_END', 'SP_BEG', 'SP_END']]

In [19]:
pdb_chain_uniprot_cancer

,PDB_ID,SP_PRIMARY,RES_BEG,RES_END,PDB_BEG,PDB_END,SP_BEG,SP_END
0,10gs_A,P09211,1,209,None,209,2,210
1,10gs_B,P09211,1,209,None,209,2,210
2,11gs_A,P09211,1,210,None,209,1,210
3,11gs_B,P09211,1,210,None,209,1,210
4,12gs_A,P09211,1,210,None,209,1,210
...,...,...,...,...,...,...,...,...
106064,8ter_T,Q92734,268,358,None,None,237,327
106065,8ttb_D,P56211,3,114,None,112,1,112
106066,8tzp_C,O95980,1,971,None,None,1,971
106067,8u89_B,Q14738,1,602,None,None,1,602


In [20]:
def check_abnormal_aa(peptide_seq):
    if not isinstance(peptide_seq, str):
        print(peptide_seq)
        raise ValueError("Input sequence must be a string.")

    len_seq = len(peptide_seq) 
    cnt = 0  
    standard_aa = ['G', 'A', 'P', 'V', 'L', 'I', 'M', 'F', 'Y', 'W', 
                   'S', 'T', 'C', 'N', 'Q', 'K', 'H', 'R', 'D', 'E']

    for i in peptide_seq:
        if i in standard_aa:
            cnt += 1

    score = float(cnt) / len_seq
    return score

def lower_chain(input_str):
    chain_list = list(input_str) 
    output_list = [] 

    for item in chain_list:
        if item.isalpha():
            a = item.lower() 
        else:
            a = item 
        output_list.append(a) 

    output_str = ''.join(output_list) 
    return output_str

## STEP 0

In [ ]:
import pandas as pd

# Step 0: parse the fasta file downloaded from the RCSB PDB
# INPUT : pdb_seqres.txt
# OUTPUT: pdb_pep_chain, pdbid_all_fasta

print("Step 0: Parsing the fasta file...")

raw_str = ''
with open('C:/Users/PREDATOR/CODING/Code_new/Data Bahan/PDB/pdb_seqres.txt', 'r') as f:
    print("Reading fasta file...")
    for line in f.readlines():
        raw_str = raw_str + line.replace('\n', '###')

print("File reading completed.")
raw_list = raw_str.split('>')
del raw_list[0]

PDB_id_lst = []
PDB_chain_lst = []
PDB_type_lst = []
PDB_seq_lst = []
PDB_seq_len_lst = []

# Loop through each sequence in the fasta file and extract relevant information
for i, x in enumerate(raw_list):
    parts = x.split('_')
    
    if len(parts) >= 2:
        PDB_id_lst.append(parts[0])
        PDB_chain = parts[1].split(' ')[0].lower()
        PDB_chain_lst.append(PDB_chain)
        
        PDB_type = x.split('mol:')
        if len(PDB_type) >= 2:
            PDB_type_lst.append(PDB_type[1].split(' ')[0])
        else:
            PDB_type_lst.append('')

        PDB_seq = x.split('###')
        if len(PDB_seq) >= 2:
            PDB_seq_lst.append(PDB_seq[1])
            PDB_seq_len_lst.append(len(PDB_seq[1]))
        else:
            PDB_seq_lst.append('')
            PDB_seq_len_lst.append(0)
    else:
        # Handle the case where the line doesn't have the expected format
        PDB_id_lst.append('')
        PDB_chain_lst.append('')
        PDB_type_lst.append('')
        PDB_seq_lst.append('')
        PDB_seq_len_lst.append(0)
    
    # Print progress every 100 iterations
    if (i + 1) % 100 == 0 or (i + 1) == len(raw_list):
        print(f"Processed {i + 1} out of {len(raw_list)} sequences.")

print("Finished parsing all sequences.")

# Create DataFrame from the extracted data
df_fasta_raw = pd.DataFrame(list(zip(PDB_type_lst, PDB_seq_len_lst, PDB_seq_lst, PDB_id_lst, PDB_chain_lst)), 
                            columns=['PDB_type', 'PDB_seq_len', 'PDB_seq', 'PDB_id', 'chain'])

# Filter for peptides (sequences <= 50 residues and type is protein)
df_fasta = df_fasta_raw[(df_fasta_raw.PDB_seq_len <= 50) & (df_fasta_raw.PDB_type == 'protein')]

# Save both the raw and filtered data to TSV files
print("Saving parsed data to files...")
df_fasta_raw.to_csv('C:/Users/PREDATOR/CODING/Code_new/Data_Hasil/Step_0/pdbid_all_fasta_all.tsv', encoding='utf-8', index=False, sep='\t')
df_fasta.to_csv('C:/Users/PREDATOR/CODING/Code_new/Data_Hasil/Step_0/pdb_pep_chain_all.tsv', encoding='utf-8', index=False, sep='\t')

print('Step 0 is finished by generating two files : pdb_pep_chain & pdbid_all_fasta!')

In [21]:
pdbid_all_fasta_all = pd.read_csv('D:/CODING/Code_new/Data_Hasil/Step_0/pdbid_all_fasta_all.tsv', sep='\t', header=0)
pdbid_all_fasta_all

,PDB_type,PDB_seq_len,PDB_seq,PDB_id,chain
0,na,10,CCGGCGCCGG,100d,a
1,na,10,CCGGCGCCGG,100d,b
2,na,12,CGCGAATTCGCG,101d,a
3,na,12,CGCGAATTCGCG,101d,b
4,protein,154,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,101m,a
...,...,...,...,...,...
825909,protein,388,MNYQPTPEDRFTFGLWTVGWQGRDPFGDATRRALDPVESVQRLAEL...,9xia,a
825910,protein,393,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...,9xim,a
825911,protein,393,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...,9xim,b
825912,protein,393,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...,9xim,c


In [22]:
pdbid_pep_chain_all = pd.read_csv('D:/CODING/Code_new/Data_Hasil/Step_0/pdb_pep_chain_all.tsv', sep='\t', header=0)
pdbid_pep_chain_all

,PDB_type,PDB_seq_len,PDB_seq,PDB_id,chain
0,protein,4,AEKA,148l,s
1,protein,11,TVPGVXTVPGV,173d,c
2,protein,11,TVPGVXTVPGV,173d,d
3,protein,8,SAXVSAXV,185d,a
4,protein,8,SAXXSAXX,193d,c
...,...,...,...,...,...
48725,protein,36,SIPPEVKFNKPFVFLMIEQNTKSPLFMGKVVNPTQK,9api,b
48726,protein,21,GIVEQCCTSICSLYQLENYCN,9ins,a
48727,protein,30,FVNQHLCGSHLVEALYLVCGERGFFYTPKA,9ins,b
48728,protein,5,XAAPL,9lpr,p


## STEP 1

In [ ]:
#!docker run --rm -v ${PWD}:/results -w /results -u $(id -u ${USER}):$(id -g ${USER}) pharmai/plip:latest -i 1vsn -yv

!docker run --rm -v /c/Users/PREDATOR/CODING/Code_new:/results -w /results pharmai/plip:latest -i 1vsn -yv

In [ ]:
import subprocess

pdb_id = '1dev'
chain = 'b'
command = f'docker run --rm -v /c/Users/PREDATOR/CODING/tesis/Data_Hasil:/results -w /results pharmai/plip:latest -i {pdb_id} --peptides {chain.upper()} --name {pdb_id}_{chain}_result'

try:
    result = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
    print(result.decode())
    print("Command executed successfully")
except subprocess.CalledProcessError as e:
    print(f"Command failed with return code {e.returncode}:")
    print(e.output.decode())

In [ ]:
def get_latest_file(directory):
    files = [f for f in os.listdir(directory) if f.endswith("_result.txt")]
    if not files:
        return None

    files.sort(reverse=True)

    # Get the latest file
    filename = files[0]
    pdb_id, chain = filename.split('_')[:2]
    return pdb_id, chain

In [ ]:
def run_plip(pdb_pep_dataset):
    df_fasta_pep = pd.read_csv(pdb_pep_dataset, sep='\t', header=0)
    df_fasta_pep = df_fasta_pep.reset_index(drop=True)
    df_predict = pd.DataFrame(columns=['pdb_id', 'pep_chain', 'predicted_chain'])
    len_df_fasta_pep = len(df_fasta_pep)

    # In case of corrupted pdb file, the last pdb file that was downloaded from the previous session will be re-downloaded
    resetmark = True
    i = 0

    # To continue from the latest file
    latest_pdb_id, latest_chain = get_latest_file("C:/Users/PREDATOR/CODING/Code_new/Data_Hasil/Step_1")

    if latest_pdb_id and latest_chain:
            print(f"Continuing after {latest_pdb_id}_{latest_chain}_result.txt")
            # Set the index to the corresponding value in df_fasta_pep
            i = df_fasta_pep[(df_fasta_pep['PDB_id'] == latest_pdb_id) & (df_fasta_pep['chain'] == latest_chain)].index[0]

    # Start loop
    while i < len_df_fasta_pep:
        pdb_id = df_fasta_pep['PDB_id'][i]
        if not isinstance(pdb_id, str):
            i+=1
            continue
        if len(pdb_id) > 5:
            i+=1
            continue

        chain = df_fasta_pep['chain'][i]

        if os.path.exists(f'/content/drive/MyDrive/Data_Hasil/Step_1/{pdb_id}_{chain}_result.txt'):
            print(f"index {i} : The file {pdb_id}_{chain}.txt already exists.")
            i+=1
            continue
        elif os.path.exists(f'/content/drive/MyDrive/Data_Hasil/Step_1/{pdb_id}.pdb'):
            if resetmark == True:
                print(f'index {i} : Reset enabled, re-downloading {pdb_id}.pdb for use in chain {chain.upper()}')
                os.system(f'plipcmd -i {pdb_id} --peptides {chain.upper()}  -vto /content/drive/MyDrive/Data_Hasil/Step_1 --name {pdb_id}_{chain}_result')
                resetmark = False
            else:
                print(f'index {i} : {pdb_id}.pdb already exists, using {pdb_id}.pdb for chain {chain.upper()}')
                os.system(f'plipcmd -f /content/drive/MyDrive/Data_Hasil/Step_1/{pdb_id}.pdb --peptides {chain.upper()}  -vto /content/drive/MyDrive/Data_Hasil/Step_1 --name {pdb_id}_{chain}_result')
        else:
            print(f"index {i} : Processing {pdb_id} chain {chain.upper()}")
            os.system(f'plipcmd -i {pdb_id} --peptides {chain.upper()}  -vto /content/drive/MyDrive/Data_Hasil/Step_1 --name {pdb_id}_{chain}_result')

        # Delete .pdb files after use
        if i + 1 < len_df_fasta_pep:
            if df_fasta_pep['PDB_id'][i+1] != df_fasta_pep['PDB_id'][i]:
                try:
                    os.remove(f'/content/drive/MyDrive/Data_Hasil/Step_1/{pdb_id}.pdb')
                    os.remove(f'/content/drive/MyDrive/Data_Hasil/Step_1/{pdb_id}_protonated.pdb')
                    prefix_to_delete = 'plipfixed.'
                    files_to_delete = glob.glob(os.path.join("C:/Users/PREDATOR/CODING/Code_new/Data_Hasil/Step_1", f"{prefix_to_delete}*"))
                    if len(files_to_delete) != 0:
                        for file_path in files_to_delete:
                            os.remove(file_path)
                except FileNotFoundError:
                    pass
        i+=1

In [ ]:
step_1 = run_plip("C:/Users/PREDATOR/CODING/Code_new/Data_Hasil/Step_0/pdb_pep_chain_all.tsv")

In [ ]:
# Step1 : Load all PDB ids that might contain peptide interaction and plip prediction results
# INPUT : pdb_pep_chain from Step 0 & analyzed file generated by PLIP (placed under ./peptide_result/ ,details in Data_construction.txt)
# OUTPUT: plip_predict_result

def load(pdb_pep_dataset, plip_result_filename):
    df_fasta_pep = pd.read_csv(pdb_pep_dataset, sep='\t', header=0)
    df_fasta_pep = df_fasta_pep.reset_index(drop=True)
    df_predict = pd.DataFrame(columns=['pdb_id', 'pep_chain', 'predicted_chain'])
    len_df_fasta_pep = len(df_fasta_pep)

    resetmark = True

    for i in range(df_fasta_pep.shape[0]):
        pdb_id = df_fasta_pep['PDB_id'][i]
        if not isinstance(pdb_id, str):
            continue
        if len(pdb_id) > 5:
            continue
        chain = df_fasta_pep['chain'][i]
        result_file_name = f'/C:/Users/PREDATOR/CODING/tesis/Data_Hasil/Step_1/{pdb_id}_{chain}_result.txt'

        try:
            for line in open(result_file_name):
                if line.startswith('Interacting chain(s):'):
                        df_predict.loc[i] = [pdb_id,chain, str(line).replace('\n','')\
                                      .replace('\r','')\
                                      .replace('Interacting chain(s):','')
                                      .lower()]
            if i % 5000 == 0:
                print('already finished files', i)
        except FileNotFoundError:
            pass

    print(df_predict)

    df_predict['predicted_chain_num'] = df_predict.predicted_chain.apply(lambda x: len(x.replace(' ', '')))
    df_predict = df_predict.loc[df_predict.predicted_chain_num > 0]
    df_predict = df_predict.drop('predicted_chain_num', axis=1)
    df_predict['predicted_chain'] = df_predict.predicted_chain.apply(lambda x: x.replace(' ', ''))
    df_predict['pep_chain'] = df_predict.pep_chain.apply(lambda x: x.replace(' ', ''))
    df_predict = df_predict.reset_index(drop=True)


    df_predict.predicted_chain = df_predict.predicted_chain.apply(lambda x: x.split(','))
    lst_col = 'predicted_chain'
    df1 = pd.DataFrame({col: np.repeat(df_predict[col].values, df_predict[lst_col].str.len())
                        for col in df_predict.columns.difference([lst_col])
                        }).assign(**{lst_col: np.concatenate(df_predict[lst_col].values)})[df_predict.columns.tolist()]

    df_predict = df1

    file_name = plip_result_filename
    df_predict.to_csv(f'/C:/Users/PREDATOR/CODING/tesis/Data_Hasil/Step_1/{file_name}.tsv', encoding='utf-8', index=False, sep='\t')
    print('finish exploding comma-separated predicted chain, successfully saved records:', df_predict.shape[0])

    print('Step 1 is finished by generating the PLIP prediction file: plip_predict_result.')

    return df_predict

In [ ]:
plip_prediction_result = load('C:/Users/PREDATOR/CODING/tesis/Data_Hasil/Step_0/pdb_pep_chain_all.tsv','plip_predict_result')

In [23]:
plip_predict_result = pd.read_csv('D:/CODING/Code_new/Data_Hasil/Step_1/plip_predict_result.tsv', sep='\t', header=0)
plip_predict_result = plip_predict_result.drop_duplicates()
plip_predict_result['predicted_chain'].str.contains(r'[A-Z0-9]', regex=True)

0        False
1        False
2        False
3        False
4        False
         ...  
75457    False
75458    False
75459    False
75460    False
75461    False
Name: predicted_chain, Length: 69983, dtype: bool

In [24]:
def find_capital_letters_or_numbers(column_data):
    return column_data.apply(lambda x: len(str(x)) > 1)

# Apply the function to the 'TextColumn' and create a new DataFrame with only True values
df_filtered = plip_predict_result[find_capital_letters_or_numbers(plip_predict_result['predicted_chain'])]
df_filtered
plip_predict_result

,pdb_id,pep_chain,predicted_chain
0,148l,s,e
1,1a07,c,a
2,1a07,c,b
3,1a07,d,b
4,1a08,c,a
...,...,...,...
75457,9api,b,a
75458,9ins,a,b
75459,9ins,b,a
75460,9lpr,p,a


In [25]:
len(plip_predict_result)

69983

In [26]:
pdb_id = '5tis'
chain = 'm'
df_fasta_pep = pd.read_csv('D:/CODING/Code_new/Data_Hasil/Step_0/pdb_pep_chain_all.tsv', sep='\t', header=0)
df_fasta_pep = df_fasta_pep.reset_index(drop=True)
i = df_fasta_pep[(df_fasta_pep['PDB_id'] == pdb_id) & (df_fasta_pep['chain'] == chain)].index[0]
print(len(df_fasta_pep))

48730


In [27]:
df_fasta_pep.shape

(48730, 5)

## STEP 2

In [28]:
# Step 2: Get fasta sequence of the predicted interacting chains
# INPUT:  pdbid_all_fasta from Step 0
# OUTPUT: -
def load_all_fasta(all_fasta_file,input_dataset): # pdbid_all_fasta # df_predict
	df_fasta = pd.read_csv(all_fasta_file,sep = '\t', header = 0)
	df_fasta_protein = df_fasta.loc[df_fasta.PDB_type=='protein']

	# df_fasta_protein['PDB_id'] = df_fasta_protein.PDB_id_chain.apply(lambda x: x.split('_')[0])
	# df_fasta_protein['chain'] = df_fasta_protein.PDB_id_chain.apply(lambda x: x.split('_')[1].lower())
	df_fasta_vocabulary = df_fasta_protein[['PDB_id','chain','PDB_seq']]

	df_predict_det = pd.merge(input_dataset,df_fasta_vocabulary,how='left',\
	                         left_on = ['pdb_id','pep_chain'],right_on = ['PDB_id','chain'])

	df_predict_det1 = pd.merge(df_predict_det,df_fasta_vocabulary,how='left',\
	                         left_on = ['pdb_id','predicted_chain'],right_on = ['PDB_id','chain'])

	print(df_predict_det1)
	df_predict_det1 =df_predict_det1.drop(['PDB_id_x','chain_x','PDB_id_y','chain_y'],axis =1)
	df_predict_det1.columns = ['pdb_id','pep_chain','predicted_chain','pep_seq','prot_seq']
	print(df_predict_det1)
	df_predict_det1['pep_seq_len'] = df_predict_det1.pep_seq.apply(lambda x: len(str(x)))
	df_predict_det1['prot_seq_len'] = df_predict_det1.prot_seq.apply(lambda x: len(str(x)))

	df_predict_det1 = df_predict_det1.dropna()

	# check sequence length(peptide<=50 & protein >50)
	df_predict_det1 = df_predict_det1.loc[(df_predict_det1.pep_seq_len <= 50) & (df_predict_det1.prot_seq_len > 50)]


	# remove records with more than 20% AA is abnormal
	df_predict_det1['peptide_seq_score'] = df_predict_det1.pep_seq.apply(lambda x: check_abnormal_aa(x))
	df_predict_det1 = df_predict_det1[df_predict_det1.peptide_seq_score >= 0.8]

	print('finish removing sequences without too many non-standard residues')
	print('-----------------------------------------------------')


	return df_predict_det1

In [29]:
pdbid_all_fasta_all = pd.read_csv('D:/CODING/Code_new/Data_Hasil/Step_0/pdbid_all_fasta_all.tsv', sep='\t', header=0)
pdbid_all_fasta_all

,PDB_type,PDB_seq_len,PDB_seq,PDB_id,chain
0,na,10,CCGGCGCCGG,100d,a
1,na,10,CCGGCGCCGG,100d,b
2,na,12,CGCGAATTCGCG,101d,a
3,na,12,CGCGAATTCGCG,101d,b
4,protein,154,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,101m,a
...,...,...,...,...,...
825909,protein,388,MNYQPTPEDRFTFGLWTVGWQGRDPFGDATRRALDPVESVQRLAEL...,9xia,a
825910,protein,393,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...,9xim,a
825911,protein,393,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...,9xim,b
825912,protein,393,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...,9xim,c


In [30]:
step_2 = load_all_fasta('D:/CODING/Code_new/Data_Hasil/Step_0/pdbid_all_fasta_all.tsv', plip_predict_result)

      pdb_id pep_chain predicted_chain PDB_id_x chain_x  \
0       148l         s               e     148l       s   
1       1a07         c               a     1a07       c   
2       1a07         c               b     1a07       c   
3       1a07         d               b     1a07       d   
4       1a08         c               a     1a08       c   
...      ...       ...             ...      ...     ...   
95451   9api         b               a     9api       b   
95452   9ins         a               b     9ins       a   
95453   9ins         b               a     9ins       b   
95454   9lpr         p               a     9lpr       p   
95455   9pai         b               a     9pai       b   

                                  PDB_seq_x PDB_id_y chain_y  \
0                                      AEKA     148l       e   
1                                      XYEX     1a07       a   
2                                      XYEX     1a07       b   
3                                  

In [31]:
step_2 = step_2.drop_duplicates()
step_2

,pdb_id,pep_chain,predicted_chain,pep_seq,prot_seq,pep_seq_len,prot_seq_len,peptide_seq_score
0,148l,s,e,AEKA,MNIFEMLRIDEGLRLKIYKDTEGYYEIGIGHLLTKSPSLNAAKSEL...,4,164,1.0
12,1a0n,a,b,PPRPLPVAPGSSKT,GSTGVTLFVALYDYEARTEDDLSFHKGEKFQILNSSEGDWWEARSL...,14,69,1.0
25,1a1m,c,a,TPYDINQML,GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...,9,278,1.0
26,1a1n,c,a,VPLRPMTY,GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...,8,276,1.0
27,1a1o,c,a,KPIVQYDNF,GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...,9,276,1.0
...,...,...,...,...,...,...,...,...
95449,8t70,p,a,TKCMIIM,MGSSHHHHHHSQDLMVTSTYIPMSQRRSWADVKPIMQDDGPNPVVP...,7,349,1.0
95450,8t70,p,b,TKCMIIM,MATEFTPSVYSLVSKPLPSNSRPSATLDEQAETEDLISQLFDLTAD...,7,520,1.0
95451,9api,b,a,SIPPEVKFNKPFVFLMIEQNTKSPLFMGKVVNPTQK,DTSHHDQDHPTFNKITPNLAEFAFSLYRQLAHQSNSTNIFFSPVSI...,36,347,1.0
95454,9lpr,p,a,XAAPL,ANIVGGIEYSINNASLCSVGFSVTRGATKGFVTAGHCGTVNATARI...,5,198,0.8


In [ ]:
step_2.to_csv('C:/Users/PREDATOR/CODING/Code_new/Data_Hasil/Step_2/step_2.tsv', encoding='utf-8', index=False, sep='\t')

In [32]:
#1
df_fasta = pd.read_csv('D:/CODING/Code_new/Data_Hasil/Step_0/pdbid_all_fasta_all.tsv',sep = '\t', header = 0)
df_fasta_protein = df_fasta.loc[df_fasta.PDB_type=='protein']
df_fasta_protein

,PDB_type,PDB_seq_len,PDB_seq,PDB_id,chain
4,protein,154,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,101m,a
7,protein,165,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSE...,102l,a
8,protein,154,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,102m,a
11,protein,167,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNSLDAAK...,103l,a
12,protein,154,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,103m,a
...,...,...,...,...,...
825909,protein,388,MNYQPTPEDRFTFGLWTVGWQGRDPFGDATRRALDPVESVQRLAEL...,9xia,a
825910,protein,393,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...,9xim,a
825911,protein,393,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...,9xim,b
825912,protein,393,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...,9xim,c


In [33]:
#2
df_fasta_vocabulary = df_fasta_protein[['PDB_id','chain','PDB_seq']]
df_fasta_vocabulary

,PDB_id,chain,PDB_seq
4,101m,a,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...
7,102l,a,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSE...
8,102m,a,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...
11,103l,a,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNSLDAAK...
12,103m,a,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...
...,...,...,...
825909,9xia,a,MNYQPTPEDRFTFGLWTVGWQGRDPFGDATRRALDPVESVQRLAEL...
825910,9xim,a,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...
825911,9xim,b,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...
825912,9xim,c,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...


## STEP 3

In [37]:
step_2 = pd.read_csv('D:/CODING/Code_new/Data_Hasil/Step_2/step_2.tsv', sep='\t')
step_2

,pdb_id,pep_chain,predicted_chain,pep_seq,prot_seq,pep_seq_len,prot_seq_len,peptide_seq_score
0,148l,s,e,AEKA,MNIFEMLRIDEGLRLKIYKDTEGYYEIGIGHLLTKSPSLNAAKSEL...,4,164,1.0
1,1a0n,a,b,PPRPLPVAPGSSKT,GSTGVTLFVALYDYEARTEDDLSFHKGEKFQILNSSEGDWWEARSL...,14,69,1.0
2,1a1m,c,a,TPYDINQML,GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...,9,278,1.0
3,1a1n,c,a,VPLRPMTY,GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...,8,276,1.0
4,1a1o,c,a,KPIVQYDNF,GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...,9,276,1.0
...,...,...,...,...,...,...,...,...
37823,8t70,p,a,TKCMIIM,MGSSHHHHHHSQDLMVTSTYIPMSQRRSWADVKPIMQDDGPNPVVP...,7,349,1.0
37824,8t70,p,b,TKCMIIM,MATEFTPSVYSLVSKPLPSNSRPSATLDEQAETEDLISQLFDLTAD...,7,520,1.0
37825,9api,b,a,SIPPEVKFNKPFVFLMIEQNTKSPLFMGKVVNPTQK,DTSHHDQDHPTFNKITPNLAEFAFSLYRQLAHQSNSTNIFFSPVSI...,36,347,1.0
37826,9lpr,p,a,XAAPL,ANIVGGIEYSINNASLCSVGFSVTRGATKGFVTAGHCGTVNATARI...,5,198,0.8


In [38]:
# Step 3: Map Uniprot ID for PDB complex by protein-chain & PDB id
# INPUT: data from Step 2 & pdb_chain_uniprot.tsv from SIFTS
# OUTPUT: UniProt_ID_list (all IDs are used as search queries on https://www.uniprot.org/uploadlists/ for unified sequences)

def map_uniprot_chain(input_dataset, pdb_chain_uniprot_file):
    df_sifts = pd.read_csv(pdb_chain_uniprot_file, sep='\t', header=0)
    df_sifts = df_sifts[['PDB', 'CHAIN', 'SP_PRIMARY']]
    df_sifts_keep = df_sifts[df_sifts['CHAIN'] != df_sifts['CHAIN']]
    df_sifts = df_sifts[df_sifts['CHAIN'] == df_sifts['CHAIN']]
    df_sifts['CHAIN'] = df_sifts.CHAIN.apply(lambda x: lower_chain(x))

    df_predict_det2 = pd.merge(input_dataset, df_sifts, how='left',
                               left_on=['pdb_id', 'predicted_chain'], 
                               right_on=['PDB', 'CHAIN'])

    df_predict_det2 = df_predict_det2.drop(['PDB', 'CHAIN'], axis=1)

    df_predict_det2_no_uni = df_predict_det2[df_predict_det2.SP_PRIMARY != df_predict_det2.SP_PRIMARY]

    df_predict_det2_no_uni = df_predict_det2_no_uni.reset_index(drop=True)

    df_predict_det2_no_uni = df_predict_det2_no_uni.drop(['prot_seq_len', 'peptide_seq_score'], axis=1)

    df_predict_det2_no_uni = df_predict_det2_no_uni[['pdb_id', 'pep_chain', 'predicted_chain', 
                                                     'pep_seq', 'pep_seq_len', 'SP_PRIMARY', 'prot_seq']]
    df_predict_det2_no_uni.rename(columns={'prot_seq': 'Sequence'}, inplace=True)

    df_predict_det3 = df_predict_det2[df_predict_det2.SP_PRIMARY == df_predict_det2.SP_PRIMARY]

    df_uni_id = df_predict_det3[['SP_PRIMARY']]
    df_uni_id.drop_duplicates(inplace=True)

    return df_predict_det2_no_uni, df_predict_det3

In [39]:
step_3_no_uni, step_3 = map_uniprot_chain(step_2,'D:/CODING/Code_new/Data Bahan/Uniprot (SIFTS)/pdb_chain_uniprot.tsv')

D:\miniconda\envs\tf\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [40]:
step_3.describe()

,pep_seq_len,prot_seq_len,peptide_seq_score
count,40577.000000,40577.000000,40577.000000
mean,23.886882,271.198659,0.988654
std,14.456775,199.550795,0.037861
min,2.000000,51.000000,0.800000
25%,11.000000,135.000000,1.000000
50%,20.000000,240.000000,1.000000
75%,37.000000,344.000000,1.000000
max,50.000000,5037.000000,1.000000


In [41]:
step_3_no_uni

,pdb_id,pep_chain,predicted_chain,pep_seq,pep_seq_len,SP_PRIMARY,Sequence
0,1acy,p,l,YNKRKRIHIGPGRAFYTTKNIIGC,24,NaN,DIVMTQSPASLVVSLGQRATISCRASESVDSYGKSFMHWYQQKPGQ...
1,1bog,c,a,GATPEDLNQKL,11,NaN,DIKMTQSPSSMYTSLGERVTITCKASQDINSFLTWFLQKPGKSPKT...
2,1ce1,p,l,GTSSPSAD,8,NaN,DIQMTQSPSSLSASVGDRVTITCKASQNIDKYLNWYQQKPGKAPKL...
3,1cu4,p,h,APKTNMKHMA,10,NaN,KVKLQQSGAELVRSGASVKLSCTASGFNIKDYYIQWVKQRPEQGLE...
4,1cu4,p,l,APKTNMKHMA,10,NaN,DVVMTQTPLSLSVTIGQPASISCKSSQSLLDSDGKTYLIWVFQRPG...
...,...,...,...,...,...,...,...
3172,8op0,b,a,PGGGSVQIVYKPKK,14,NaN,GAMAEVQLQASGGVFVQSGGSLRLSCAASGATSTFDGMGWFRQAPG...
3173,8opi,b,a,PGGGSVQIVYKPKK,14,NaN,GAMAEVQLQASGGVFVQSGGSLRLSCAASGATSTFDGMGWFRQAPG...
3174,8pii,b,a,PGGGSVQIVYKPKK,14,NaN,GAMAEVQLQASGGVFVQSGGSLRLSCAASGATSTFDGMGWFRQAPG...
3175,8shi,c,h,VRSRRALRL,9,NaN,MGVTQTPKFQVLKTGQSMTLQCAQDMNHEYMSWYRQDPGMGLRLIH...


In [42]:
step_3

,pdb_id,pep_chain,predicted_chain,pep_seq,prot_seq,pep_seq_len,prot_seq_len,peptide_seq_score,SP_PRIMARY
0,148l,s,e,AEKA,MNIFEMLRIDEGLRLKIYKDTEGYYEIGIGHLLTKSPSLNAAKSEL...,4,164,1.0,P00720
1,1a0n,a,b,PPRPLPVAPGSSKT,GSTGVTLFVALYDYEARTEDDLSFHKGEKFQILNSSEGDWWEARSL...,14,69,1.0,P06241
2,1a1m,c,a,TPYDINQML,GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...,9,278,1.0,P01889
3,1a1n,c,a,VPLRPMTY,GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...,8,276,1.0,P01889
4,1a1o,c,a,KPIVQYDNF,GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...,9,276,1.0,P01889
...,...,...,...,...,...,...,...,...,...
43749,8t70,p,a,TKCMIIM,MGSSHHHHHHSQDLMVTSTYIPMSQRRSWADVKPIMQDDGPNPVVP...,7,349,1.0,J9VSJ6
43750,8t70,p,b,TKCMIIM,MATEFTPSVYSLVSKPLPSNSRPSATLDEQAETEDLISQLFDLTAD...,7,520,1.0,T2BPA1
43751,9api,b,a,SIPPEVKFNKPFVFLMIEQNTKSPLFMGKVVNPTQK,DTSHHDQDHPTFNKITPNLAEFAFSLYRQLAHQSNSTNIFFSPVSI...,36,347,1.0,P01009
43752,9lpr,p,a,XAAPL,ANIVGGIEYSINNASLCSVGFSVTRGATKGFVTAGHCGTVNATARI...,5,198,0.8,P00778


In [43]:
step_3 = step_3.drop_duplicates().dropna()
step_3

,pdb_id,pep_chain,predicted_chain,pep_seq,prot_seq,pep_seq_len,prot_seq_len,peptide_seq_score,SP_PRIMARY
0,148l,s,e,AEKA,MNIFEMLRIDEGLRLKIYKDTEGYYEIGIGHLLTKSPSLNAAKSEL...,4,164,1.0,P00720
1,1a0n,a,b,PPRPLPVAPGSSKT,GSTGVTLFVALYDYEARTEDDLSFHKGEKFQILNSSEGDWWEARSL...,14,69,1.0,P06241
2,1a1m,c,a,TPYDINQML,GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...,9,278,1.0,P01889
3,1a1n,c,a,VPLRPMTY,GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...,8,276,1.0,P01889
4,1a1o,c,a,KPIVQYDNF,GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...,9,276,1.0,P01889
...,...,...,...,...,...,...,...,...,...
43749,8t70,p,a,TKCMIIM,MGSSHHHHHHSQDLMVTSTYIPMSQRRSWADVKPIMQDDGPNPVVP...,7,349,1.0,J9VSJ6
43750,8t70,p,b,TKCMIIM,MATEFTPSVYSLVSKPLPSNSRPSATLDEQAETEDLISQLFDLTAD...,7,520,1.0,T2BPA1
43751,9api,b,a,SIPPEVKFNKPFVFLMIEQNTKSPLFMGKVVNPTQK,DTSHHDQDHPTFNKITPNLAEFAFSLYRQLAHQSNSTNIFFSPVSI...,36,347,1.0,P01009
43752,9lpr,p,a,XAAPL,ANIVGGIEYSINNASLCSVGFSVTRGATKGFVTAGHCGTVNATARI...,5,198,0.8,P00778


# STEP 4

In [44]:
# Step 4: Load Uniport sequences and family information & filter out MHC families
# INPUT: the data from Step 3 & uniprot2seq from UniProt Website (a tab separated file with fields including Uniprot_id,Uniprot Sequence,Protein_name,Protein_families)
# OUTPUT: interacted peptide-protein pairs from PDB (a '#' separated file with fields including pdb_id,pep_chain,prot_chain,pep_seq,Uniprot_id,prot_seq,protein_families)

import pandas as pd
def load_uni_seq(df_predict_det2_no_uni, input_dataset, uniprot2seq_file):
    df_uni2seq = pd.read_csv(uniprot2seq_file, sep='\t', header=0)
    df_uni2seq.rename(columns={'Entry': 'Uniprot_id',
                               'Protein names': 'Protein_name',
                               'Protein families': 'Protein_families'}, inplace=True)

    df_uni2seq = df_uni2seq.drop_duplicates(['Uniprot_id', 'Sequence'], keep='first')
    df_uni2seq = df_uni2seq.fillna('Unknown_from_uniprot')

    df_predict_det4 = pd.merge(input_dataset, df_uni2seq, how='left', 
                               left_on=['SP_PRIMARY'], right_on=['Uniprot_id'])

    df_predict_det4 = df_predict_det4.drop(
        ['Uniprot_id', 'Protein_name', 'prot_seq', 'prot_seq_len', 'peptide_seq_score'], axis=1)

    df_predict_det4 = df_predict_det4.drop_duplicates(
        ['pdb_id', 'pep_seq', 'SP_PRIMARY', 'Sequence'], keep='first')

    df_predict_det2_no_uni.rename(columns={'prot_seq': 'Sequence'}, inplace=True)
    df_predict_det2_no_uni['Protein_families'] = 'Unknown Uniprot_ids'

    df_predict_det6 = pd.concat([df_predict_det2_no_uni, df_predict_det4], ignore_index=True)

    df_predict_det6['plip_prot_chain'] = df_predict_det6.predicted_chain.apply(lambda x: x.upper())

    df_predict_det6 = df_predict_det6.drop_duplicates(['pep_seq', 'Sequence'], keep='first')

    df_predict_det6 = df_predict_det6.reset_index(drop=True)

    df_predict_det6['prot_seq_len'] = df_predict_det6.Sequence.apply(lambda x: len(str(x)))
    df_predict_det6 = df_predict_det6[df_predict_det6.prot_seq_len <= 5000]

    df_pdb_pairs = df_predict_det6[['pdb_id', 'pep_chain', 'plip_prot_chain', 
                                    'pep_seq', 'SP_PRIMARY', 'Sequence', 'Protein_families']]
    df_pdb_pairs.columns = ['pdb_id', 'pep_chain', 'prot_chain', 
                            'pep_seq', 'Uniprot_id', 'prot_seq', 'protein_families']

    file_name = 'train_pairs_pdb'
    df_pdb_pairs.to_csv(file_name, encoding='utf-8', index=False, sep='#')

    return df_pdb_pairs

In [45]:
# Memanggil fungsi load_uni_seq
step_4 = load_uni_seq(step_3_no_uni, step_3, 
                      'D:/CODING/Code_new/Data Bahan/uniprot2seq_cancer_from_uniprot.tsv')

# Membersihkan data hasil dari load_uni_seq
step_4 = step_4.dropna().drop_duplicates()
step_4 = pd.read_csv('D:/CODING/Code_new/Data_Hasil/Step_4/step_4.tsv', sep='\t')

In [46]:
step_4

,pdb_id,pep_chain,prot_chain,pep_seq,Uniprot_id,prot_seq,protein_families
0,1a0n,a,B,PPRPLPVAPGSSKT,P06241,MGCVQCKDKEATKLTEERDGSLNQSSGYRYGTDPTPQHYPSFGVTS...,"Protein kinase superfamily, Tyr protein kinase..."
1,1a1m,c,A,TPYDINQML,P01889,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,Unknown_from_uniprot
2,1a1n,c,A,VPLRPMTY,P01889,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,Unknown_from_uniprot
3,1a1o,c,A,KPIVQYDNF,P01889,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,Unknown_from_uniprot
4,1a6a,c,A,PVSKMRMATPLLMQA,P01903,MAISGVPVLGFFIIAVLMSAQESWAIKEEHVIIQAEFYLNPDQSGE...,MHC class II family
...,...,...,...,...,...,...,...
3272,8jzv,b,A,TWEADDVDDDLLYQACDDIERLTQ,P27694,MVGQLSEGAIAAIMQKGDTNIKPILQVINIRPITTGNSPPRYRLLM...,Replication factor A protein 1 family
3273,8jzy,b,A,DDFANDDIDSYMIAMETTI,P27694,MVGQLSEGAIAAIMQKGDTNIKPILQVINIRPITTGNSPPRYRLLM...,Replication factor A protein 1 family
3274,8k00,b,A,GTSSGSAFSADDLMSIDLAEQMANDSDDSIS,P27694,MVGQLSEGAIAAIMQKGDTNIKPILQVINIRPITTGNSPPRYRLLM...,Replication factor A protein 1 family
3275,8oep,b,A,RQERLQRRRETQV,P26045,MTSRLRALGGRINNIRTSELPKEKTRSEVICSIHFLDGVVQTFKVT...,"Protein-tyrosine phosphatase family, Non-recep..."


In [47]:
# 5. **Menggabungkan Data dengan Peptida dari DrugBank**
db_peptides['peptide_seq_score'] = db_peptides.dg_sequence.apply(lambda x: check_abnormal_aa(x))
db_peptides = db_peptides[db_peptides.peptide_seq_score >= 0.8]
db_peptides = db_peptides[['dg_id', 'dg_sequence', 'target_uniprot']]

D:\miniconda\envs\tf\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [48]:
cancer_proteins_uniprot = cancer_proteins_uniprot[['Entry', 'Protein families', 'Sequence']]

db_peptides_final = pd.merge(db_peptides, cancer_proteins_uniprot, 
                             left_on='target_uniprot', right_on='Entry', how='left')
db_peptides_final = db_peptides_final.dropna(subset=['Sequence'])
db_peptides_final.insert(1, 'pep_chain', float('nan'))
db_peptides_final.insert(2, 'prot_chain', float('nan'))

db_peptides_final = db_peptides_final.rename(columns={'dg_id': 'pdb_id',
                                                      'dg_sequence': 'pep_seq',
                                                      'target_uniprot': 'Uniprot_id',
                                                      'Protein families': 'protein_families',
                                                      'Sequence': 'prot_seq'})

db_peptides_final = db_peptides_final[['pdb_id', 'pep_chain', 'prot_chain', 
                                       'pep_seq', 'Uniprot_id', 'prot_seq', 'protein_families']]
db_peptides_final.reset_index(drop=True, inplace=True)

In [49]:
step_4_with_drugbank = pd.concat([step_4, db_peptides_final], ignore_index=True)

In [50]:
step_4_with_drugbank

,pdb_id,pep_chain,prot_chain,pep_seq,Uniprot_id,prot_seq,protein_families
0,1a0n,a,B,PPRPLPVAPGSSKT,P06241,MGCVQCKDKEATKLTEERDGSLNQSSGYRYGTDPTPQHYPSFGVTS...,"Protein kinase superfamily, Tyr protein kinase..."
1,1a1m,c,A,TPYDINQML,P01889,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,Unknown_from_uniprot
2,1a1n,c,A,VPLRPMTY,P01889,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,Unknown_from_uniprot
3,1a1o,c,A,KPIVQYDNF,P01889,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,Unknown_from_uniprot
4,1a6a,c,A,PVSKMRMATPLLMQA,P01903,MAISGVPVLGFFIIAVLMSAQESWAIKEEHVIIQAEFYLNPDQSGE...,MHC class II family
...,...,...,...,...,...,...,...
3708,DB11995,NaN,NaN,APSNLAS,Q9UNQ0,MSSSNVEVFIPVSQGNTNGFPATASNDLKAFTEGAVLSFHNICYRV...,"ABC transporter superfamily, ABCG family, Eye ..."
3709,DB11995,NaN,NaN,QQWSFNPPT,Q9UNQ0,MSSSNVEVFIPVSQGNTNGFPATASNDLKAFTEGAVLSFHNICYRV...,"ABC transporter superfamily, ABCG family, Eye ..."
3710,DB11995,NaN,NaN,GYTFTSYNMH,Q9UNQ0,MSSSNVEVFIPVSQGNTNGFPATASNDLKAFTEGAVLSFHNICYRV...,"ABC transporter superfamily, ABCG family, Eye ..."
3711,DB11995,NaN,NaN,AIYPGNGDTSYNQKFKG,Q9UNQ0,MSSSNVEVFIPVSQGNTNGFPATASNDLKAFTEGAVLSFHNICYRV...,"ABC transporter superfamily, ABCG family, Eye ..."


In [52]:
step_4 = pd.read_csv('D:/CODING/Code_new/Data_Hasil/Step_4_with_drugbank/step_4_with_drugbank.tsv', sep='\t')

In [45]:
step_4

,pdb_id,pep_chain,prot_chain,pep_seq,Uniprot_id,prot_seq,protein_families
0,1a0n,a,B,PPRPLPVAPGSSKT,P06241,MGCVQCKDKEATKLTEERDGSLNQSSGYRYGTDPTPQHYPSFGVTS...,"Protein kinase superfamily, Tyr protein kinase..."
1,1a1m,c,A,TPYDINQML,P01889,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,Unknown_from_uniprot
2,1a1n,c,A,VPLRPMTY,P01889,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,Unknown_from_uniprot
3,1a1o,c,A,KPIVQYDNF,P01889,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,Unknown_from_uniprot
4,1a6a,c,A,PVSKMRMATPLLMQA,P01903,MAISGVPVLGFFIIAVLMSAQESWAIKEEHVIIQAEFYLNPDQSGE...,MHC class II family
...,...,...,...,...,...,...,...
3708,DB11995,NaN,NaN,APSNLAS,Q9UNQ0,MSSSNVEVFIPVSQGNTNGFPATASNDLKAFTEGAVLSFHNICYRV...,"ABC transporter superfamily, ABCG family, Eye ..."
3709,DB11995,NaN,NaN,QQWSFNPPT,Q9UNQ0,MSSSNVEVFIPVSQGNTNGFPATASNDLKAFTEGAVLSFHNICYRV...,"ABC transporter superfamily, ABCG family, Eye ..."
3710,DB11995,NaN,NaN,GYTFTSYNMH,Q9UNQ0,MSSSNVEVFIPVSQGNTNGFPATASNDLKAFTEGAVLSFHNICYRV...,"ABC transporter superfamily, ABCG family, Eye ..."
3711,DB11995,NaN,NaN,AIYPGNGDTSYNQKFKG,Q9UNQ0,MSSSNVEVFIPVSQGNTNGFPATASNDLKAFTEGAVLSFHNICYRV...,"ABC transporter superfamily, ABCG family, Eye ..."
